In [ ]:
!pip install numpy imageio matplotlib

In [ ]:
import OpenEXR
import Imath
import numpy as np


def exr2numpy(exr):
    """ converts 1-channel exr-data to 2D numpy arrays """                                                                    
    file = OpenEXR.InputFile(exr)
    header = file.header()
    print (header)
    channels = ["R", "G", "B"]
    if "A" in header["channels"]:
        channels.append("A")
    
    # Compute the size
    dw = file.header()["dataWindow"]
    sz = (dw.max.x - dw.min.x + 1, dw.max.y - dw.min.y + 1)

    # Read the channels as 32-bit floats
    image = []
    for channel in channels:
        FLOAT = Imath.PixelType(Imath.PixelType.FLOAT)
        data_str = file.channel(channel, FLOAT)
        data = np.frombuffer(data_str, dtype = np.float32)
        data.shape = (sz[1], sz[0])
        image.append(data.copy())
    image = np.stack(image, axis=-1).astype(np.float32)
    return image

In [ ]:
# exr2numpy("/Users/ruilongli/workspace/blenderlib/results_sv/Hare_male_full_RM/Idle_2/cam_0000000.exr")

In [ ]:
import json
import os

import imageio
import numpy as np


class SubjectParser:
    """Single subject data parser."""

    WIDTH = 400
    HEIGHT = 400

    def __init__(self, root_fp: str):
        # an example of `root_fp`:
        # /Users/ruilongli/workspace/blenderlib/results_sv/Hare_male_full_RM/Idle_2
        self.root_fp = root_fp
        self.root_dir = os.path.join(root_fp)
        self._camera_ids = sorted([
            file
            for file in os.listdir(self.root_fp) if "cam" in file
        ])
        self._frame_ids = sorted([
            int(file.replace(".png", "").replace("image", ""))
            for file in os.listdir(os.path.join(self.root_fp, self._camera_ids[0])) 
            if "image" in file
        ])

    @property
    def camera_ids(self):
        return self._camera_ids

    @property
    def frame_ids(self):
        return self._frame_ids

    def load_camera(self, frame_id, camera_id):
        path = os.path.join(self.root_dir, "%s.json" % camera_id)
        with open(path, mode="r") as fp:
            data = json.load(fp)
            intrin = data[str(frame_id)]["intrin"]
            extrin = data[str(frame_id)]["extrin"]
            K = np.array(intrin, dtype=np.float32)
            c2w = np.linalg.inv(np.array(extrin, dtype=np.float32))
            return K, c2w  # shape [3, 3], [4, 4]

    def load_image(self, frame_id, camera_id):
        path = os.path.join(
            self.root_dir,
            camera_id,
            "image%04d.png" % frame_id,
        )
        image = imageio.imread(path)
        return image  # shape [HEIGHT, WIDTH, 4], value 0 ~ 255

    def load_depth(self, frame_id, camera_id):
        # The 3 channels are supposed to be the same.
        path = os.path.join(
            self.root_dir,
            camera_id,
            "depth%04d.exr" % frame_id,
        )
        image = exr2numpy(path)
        return image  # shape [HEIGHT, WIDTH, 3]
    
    def load_flow(self, frame_id, camera_id):
        # The 4 channels are 2D vectors giving the motion towards 
        # the next and previous frame position in pixel space.
        # https://docs.blender.org/manual/en/latest/render/layers/passes.html
        path = os.path.join(
            self.root_dir,
            camera_id,
            "flow%04d.exr" % frame_id,
        )
        image = exr2numpy(path)
        return image  # shape [HEIGHT, WIDTH, 4]

    def load_meta_data(self, action, frame_ids=None):
        fp = os.path.join(self.root_dir, action, "meta_data.npz")
        data = np.load(fp, allow_pickle=True)
        keys = [
            "rest_matrixs",
            "rest_tails",
            "lbs_weights",
            "rest_verts",
            "faces",
            "pose_matrixs",
            "pose_verts",
            "pose_tails",
        ]
        return {
            key: (
                data[key][frame_ids]
                if (frame_ids is not None and "pose_" in key)
                else data[key]
            )
            for key in keys
        }

In [ ]:
def flow_visualize(flow, max_range = 1e3):
    """ Original code from SINTEL toolbox, by Jonas Wulff.
    """
    import matplotlib.colors as colors
    du = flow[:, :, 0]
    dv = flow[:, :, 1]
    [h,w] = du.shape
    max_flow = min(max_range, np.max(np.sqrt(du * du + dv * dv)))
    img = np.ones((h, w, 3), dtype=np.float64)
    # angle layer
    img[:, :, 0] = (np.arctan2(dv, du) / (2 * np.pi) + 1) % 1.0
    # magnitude layer, normalized to 1
    img[:, :, 1] = np.sqrt(du * du + dv * dv) / (max_flow + 1e-8)
    # phase layer
    #img[:, :, 2] = valid
    # convert to rgb
    img = colors.hsv_to_rgb(img)
    # remove invalid point
    img[:, :, 0] = img[:, :, 0]
    img[:, :, 1] = img[:, :, 1]
    img[:, :, 2] = img[:, :, 2]
    return img

In [ ]:
parser = SubjectParser(
    root_fp = "/Users/ruilongli/workspace/blenderlib/results_sv/Hare_male_full_RM/Idle_2",
)
# frame_id = parser.frame_ids[10]
# camera_id = parser.camera_ids[0]

# image = parser.load_image(frame_id, camera_id)
# depth = parser.load_depth(frame_id, camera_id)
# flow = parser.load_flow(frame_id, camera_id)
# print (image.shape, depth.shape, flow.shape)

In [ ]:
import matplotlib.pyplot as plt
import cv2

camera_id = parser.camera_ids[1]
frame_ids = parser.frame_ids[2:4]
print (frame_ids)

images = [
    parser.load_image(frame_id, camera_id)
    for frame_id in frame_ids
]
flow = parser.load_flow(frame_ids[0], camera_id)
flow[..., 2:4] *= images[0][..., -1:] / 255.0

print (flow[...,  2:4].max(), flow[..., 2:4].min())
fx, fy = flow[51, 177, 2:4]
print (fx, fy)
print (flow[51, 173, 2:4], images[0][51, 173, :])

cv2.circle(images[0], (173, 51), 3, (255, 0, 0, 255), -1)
plt.imshow(np.uint8(images[0]))
plt.show()

cv2.circle(images[1], (int(177 - fx), int(51 + fy)), 3, (255, 0, 0, 255), -1)
plt.imshow(np.uint8(images[1]))
plt.show()

print (
    np.where(np.linalg.norm(flow, axis=-1) == np.linalg.norm(flow, axis=-1).max())
)

plt.imshow(np.uint8(images[1] / 2. + images[0] / 2.))
plt.show()

images = [
    parser.load_image(frame_id, camera_id)
    for frame_id in frame_ids
]

In [ ]:
# plt.imshow(np.abs(flow[..., 0]))
# plt.show()
# plt.imshow(np.abs(flow[..., 1]))
# plt.show()
# plt.imshow(np.abs(flow[..., 2]))
# plt.show()
# plt.imshow(np.abs(flow[..., 3]))
# plt.show()


In [ ]:
plt.imshow(flow_visualize(flow[..., 0:2]))
plt.show()

In [ ]:
import cv2

x, y = np.meshgrid(
    np.arange(flow.shape[1], dtype=np.float32),
    np.arange(flow.shape[0], dtype=np.float32),
    indexing="xy",
)
print (x.shape, y.shape, flow.shape)
image_remap = cv2.remap(
    images[1], 
    x - flow[..., 2], 
    y + flow[..., 3],
    cv2.INTER_LINEAR,
) * (images[0][..., -1:] / 255.0)
print (image_remap[51, 173, :])

plt.imshow(
    np.uint8(image_remap / 2. + images[0] / 2.)
)
plt.show()
